In [1]:
import math
import time
import random
import copy

## Problema Dormitórios

In [2]:
dormitorios = ['sao paulo', 'flamengo', 'coritiba', 'cruzeiro', 'fortaleza']

preferencias = {
    'amanda' : ('cruzeiro', 'coritiba'),
    'pedro' : ('sao paulo', 'fortaleza'),
    'marcos' : ('flamengo', 'sao paulo'),
    'priscila' : ('sao paulo', 'fortaleza'),
    'jessica' : ('flamengo', 'cruzeiro'),
    'paulo' : ('coritiba', 'fortaleza'),
    'fred' : ('fortaleza', 'flamengo'),
    'suzana' : ('cruzeiro', 'coritiba'),
    'laura' : ('cruzeiro', 'coritiba'),
    'ricardo' : ('coritiba', 'flamengo')
}

In [3]:
dominio = [(0, (len(dormitorios) * 2) - i - 1) for i in range(len(dormitorios) * 2)]
dominio

[(0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [4]:
def print_vagas(solucao:list):
    vagas = []

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o numero do indice do dormitorio
        dormitorio = dormitorios[vagas[solucao[i]]]
        print(list(preferencias.keys())[i], dormitorio)
        # apos preencher a vaga, retira ela
        # limpa a lista de vagas
        del vagas[solucao[i]]

print_vagas(solucao=[6,1,2,1,2,0,2,2,0,0])

amanda cruzeiro
pedro sao paulo
marcos flamengo
priscila flamengo
jessica coritiba
paulo sao paulo
fred fortaleza
suzana fortaleza
laura coritiba
ricardo cruzeiro


In [5]:
def custo_dormitorio(solucao:list):
    vagas = []
    custo = 0

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o dormitorio na posicao i
        dormitorio = dormitorios[vagas[solucao[i]]]
        # recupera as preferencias da pessoa atual na posicao i
        pref = preferencias.get(list(preferencias.keys())[i])
        
        # primeiro quarto desejado
        if dormitorio == pref[0]:
            pass
        # segunda preferencia + 1 pen
        elif dormitorio == pref[1]:
            custo += 1
        # nenhuma das duas + 3 pen
        else:
            custo += 3

        del vagas[solucao[i]]

    return custo


custo_dormitorio(solucao=[6,1,2,1,2,0,2,2,0,0])

16

## Busca randômica

In [6]:
def busca_randomica(dominio, funcao_custo, numero_solucao):
    melhor_custo = 999
    melhor_solucao = None

    for i in range(numero_solucao):
        # gera uma solucao randomica
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]

        # calcula os custos
        custo_solucao = funcao_custo(solucao)

        # verifica se a solucao foi melhor
        if custo_solucao < melhor_custo:
            melhor_custo = custo_solucao
            melhor_solucao = solucao

    return melhor_solucao

custo_dormitorio(busca_randomica(dominio=dominio, funcao_custo=custo_dormitorio, numero_solucao=1000))


6

## Hill Climb

In [7]:
def hill_climb(dominio, funcao_custo):
    # gera uma solucao randomica
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo, numero_solucao=1000)

    while True:
        vizinhos = []

        for i in range(len(dominio)):
            # verifica se o numero é menor que o valor maximo
            if solucao[i] < dominio[i][1]:
                # copia a solucao
                novo_vizinho = copy.deepcopy(solucao)
                # modifica o vizinho na posicao i em +1
                novo_vizinho[i] = novo_vizinho[i] + 1
                vizinhos.append(novo_vizinho)
            
            # verifica se o valor é maior que o menor valor
            if solucao[i] > dominio[i][0]:
                novo_vizinho = copy.deepcopy(solucao)
                # modifica o vizinho na posicao i em -1
                novo_vizinho[i] = novo_vizinho[i] - 1

        # verifica o custo da soluca
        atual = funcao_custo(solucao)
        # adiciona 
        melhor = atual

        for i in range(len(vizinhos)):
            custo = funcao_custo(vizinhos[i])

            # substitui com as solucoes melhores
            if custo < melhor:
                melhor = custo
                solucao = vizinhos[i]

        # melhor solucao achada
        if melhor == atual:
            break

    return solucao

custo_dormitorio(hill_climb(dominio=dominio, funcao_custo=custo_dormitorio))

5

## Simulated Annealing

In [8]:
from jinja2 import FunctionLoader


def simulated_annealing(dominio, funcao_custo, temperatura = 10000.0, taxa_resfriamento = 0.95, passo = 1):
    # gera uma solucao aleatoria
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo, numero_solucao=1000)

    # irá rodar até resfriar
    while temperatura > 0.1:
        # escolhe um index randomico para modificar
        random_idx = random.randint(0, len(dominio) -1)
        
        # escohe a direção
        direcao = random.randint(-passo, passo)

        # copia a solucao para modificar
        solucao_mod = copy.deepcopy(solucao)
        solucao_mod[random_idx] += direcao

        # se for menor que a permitido, substitui pelo menor permitido
        if solucao_mod[random_idx] < dominio[random_idx][0]:
            solucao_mod[random_idx] = dominio[random_idx][0]
        # se for maior que a permitido, substitui pelo meior permitido
        elif solucao_mod[random_idx] > dominio[random_idx][1]:
            solucao_mod[random_idx] = dominio[random_idx][1]
        
        # calculo dos custos
        custo_solucao = funcao_custo(solucao)
        custo_solucao_mod = funcao_custo(solucao_mod)

        # calcula a probabilidade de escolher uma solucao 'pior'
        # prob = 0 até 1
        prob = pow(math.e, (-custo_solucao_mod - custo_solucao) / temperatura)

        if custo_solucao_mod < custo_solucao or random.random() < prob:
            solucao = solucao_mod

        # dominui a temperatura
        temperatura *= taxa_resfriamento
    
    return solucao

custo_dormitorio(simulated_annealing(dominio=dominio, funcao_custo=custo_dormitorio))

14

## Algoritmo Genético

### Mutação

In [11]:
def mutacao(dominio, passo, solucao:list):
    # pega um index aleatorio para fazer a mutacao
    random_idx = random.randint(0, len(solucao) - 1)

    solucao_mutante = copy.deepcopy(solucao)
    
    # 50% de chance de mutar em +1 ou -1
    if random.random() < 0.5:
        # verifica se é possivel diminuir uma quantidade :passo do numero escolhido
        if (solucao_mutante[random_idx] - passo) >= dominio[random_idx][0]:
            solucao_mutante[random_idx] -= passo

    else:
        # verifica se é possivel diminuir uma quantidade :passo do numero escolhido
        if (solucao_mutante[random_idx] + passo) <= dominio[random_idx][1]:
            solucao_mutante[random_idx] += passo

    return solucao_mutante

# teste
mutacao(dominio=dominio, passo=1, solucao=[6,1,2,1,2,0,2,2,0,0])

[6, 1, 2, 1, 2, 0, 3, 2, 0, 0]

### Cruzamento

In [55]:
def cruzamento(dominio, individuo1, individuo2):
    # escolhe o ponto de corte aleatoriamente
    ponto_corte = random.randint(1, len(dominio) -1)

    return individuo1[0:ponto_corte] + individuo2[ponto_corte:]

cruzamento(dominio=dominio, individuo1=[6,1,2,1,2,0,2,2,0,0], individuo2=[5,0,3,0,2,0,2,2,6,1])

[6, 1, 3, 0, 2, 0, 2, 2, 6, 1]

### GA

In [60]:
def genetico(dominio, funcao_custo, tamanha_populacao = 50, passo = 1,
             prob_mutacao = 0.2, elitismo = 0.2, numero_geracao= 100):
    # gera uma populacao aleatoria de acordo com o :tamanha_populacao
    populacao = []
    for i in range(tamanha_populacao):
        populacao.append(busca_randomica(dominio=dominio, funcao_custo=funcao_custo, numero_solucao=1000))

    # numero de individuos que serao selecionados como pais
    numero_elitismo = int(elitismo * tamanha_populacao)

    for _ in range(numero_geracao):
        # lista com individuos e seu custo
        custos = [(funcao_custo(ind), ind) for ind in populacao]
        # organiza de acordo com o melhor custo
        custos.sort()

        # seleciona somente os individuos ordenados de acordo com seus custos
        individuos_ordenados = [ind for (_, ind) in custos]

        # pega a relacao de melhores individuos
        populacao = individuos_ordenados[0:numero_elitismo]

        while len(populacao) < tamanha_populacao:
            # testa a probabilidade de mutacao ou cruzamento
            # mutacao
            if random.random() < 0.5:
                # seleciona um pai aleatorio
                mutacao_idx = random.randint(0, numero_elitismo)
                # adiciona o individuo mutado na populacao
                populacao.append(mutacao(dominio=dominio, passo=passo, solucao=individuos_ordenados[mutacao_idx]))
            # cruzamento
            else:
                # seleciona dois individuos para mutacao
                ind1 = random.randint(0, numero_elitismo)
                ind2 = random.randint(0, numero_elitismo)

                # verifica se os individuos sao iguais
                if ind2 == ind1:
                    # se for, acha um diferente
                    while ind2 == ind1:
                        ind2 = random.randint(0, numero_elitismo)

                populacao.append(cruzamento(dominio=dominio, individuo1=individuos_ordenados[ind1], individuo2=individuos_ordenados[ind2]))

    return custos[0]

genetico(dominio=dominio, funcao_custo=custo_dormitorio)

(2, [4, 0, 1, 0, 0, 3, 3, 1, 1, 0])